<a href="https://colab.research.google.com/github/quila-g1/ML_model_for_Text_Analysis/blob/main/TrainingModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk
!pip install transformers
!pip install torch
!pip install chardet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.0 MB/s eta 0:00:00


In [2]:
d#import files needed #download punkt and stopwords
import re
import nltk
nltk.download()
from transformers import pipeline
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt


      Unzipping tokenizers/punkt.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords


      Unzipping corpora/stopwords.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


In [3]:
#put text
text = '''I have several small dogs and for the most part they like the BestFriends Beds. HOWEVER they only like the shag. NONE of then will sleep in the Mink. Plus ... first washing (which unfortunately was beyond return window) the seams came apart in several places. Theyre a no go for me '''

In [4]:
#change all words to lowercase
text = text.lower()

In [5]:
#tokenize the text and remove all the unnecessary words.
from nltk import word_tokenize
tokenizedWords = word_tokenize(text)


In [6]:
#stopwords to remove the stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

filtered_words = [word for word in tokenizedWords if word not in stop_words]


In [7]:
#stem the filteredWords from the tokenized words
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
stemmed = [porter.stem(word) for word in filtered_words]
testCase = " ".join(stemmed)
print(testCase)

sever small dog part like bestfriend bed . howev like shag . none sleep mink . plu ... first wash ( unfortun beyond return window ) seam came apart sever place . theyr go


In [8]:
#first we will go through the sarcasm model if there is any sarcasm or irony on the text
task='irony'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


In [9]:
def preprocess(text):
    new_text = [
    ]
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

te = preprocess(testCase)
encoded_input = tokenizer(te, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) non_irony 0.9716
2) irony 0.0284


In [17]:
answer = pipeline('sentiment-analysis')(testCase)
print(answer[0])
sent = list()
for key, value in answer[0].items():
  sent.append(value)


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'label': 'NEGATIVE', 'score': 0.9860025644302368}
NEGATIVE


In [211]:
tokenizer = AutoTokenizer.from_pretrained("LiYuan/amazon-review-sentiment-analysis")

model = AutoModelForSequenceClassification.from_pretrained("LiYuan/amazon-review-sentiment-analysis")

second = preprocess(testCase)
encoded_input2 = tokenizer(second, return_tensors='pt')
output2 = model(**encoded_input2)
print(output2)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.7708, -1.1863, -0.6974,  0.4267,  1.7541]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [200]:
prediction = output2.logits.detach().numpy()
pred= softmax(prediction)
print(pred)

[[0.6012771  0.10354435 0.04677499 0.04958132 0.19882214]]


In [175]:
if labels[ranking[1]] == "irony" and scores[ranking[1]] >= 0.29:
  list1 =np.array(pred)
  list2 = np.sort(list1, axis=None)
  counter = 0
  for i in list1[0]:
    counter = counter + 1
    if i == list2[3]:
      print(counter)
      break
  if counter >= 3:
    print("positive")
  else
    print("negative")
else
  if labels[ranking[1]] == "irony" and scores[ranking[1]] <= 0.05 and sent[0] == 'NEGATIVE' and sent[1]> 0.96:
    print("negative")
  elif labels[ranking[1]] == "irony" and scores[ranking[1]] <= 0.05 and sent[0] == 'POSITIVE' and sent[1]> 0.96:
    print("positive")
  elif sent[0] == 'NEGATIVE' and sent[1]< 0.50:
    list1 =np.array(pred)
    list2 = np.sort(list1, axis=None)
    counter = 0
    if list2[4]>=3:
      print("positive")
    else
      print("negative")
  else
    list1 = np.array(pred)
    list2 = np.sort(list1, axis=None)
    counter = 0
    if list2[4]>=3:
      print("positive")
    else
      print("negative")

5
